# VTTI Dataset Image Preprocessing
- Problem: DSO expects undistored image frames
- Goals:
    - Use .yaml data to undistort image
    - Update camera intrinsics for simple Pinhole calibration

In [1]:
import cv2
import numpy as np
import os
import glob
import yaml

# SAVE YOUR WORK
%load_ext autoreload
%autoreload 2
%autosave 180

Autosaving every 180 seconds


In [2]:
# Load in yaml file 
with open('/home/daniel-choate/ASAR/s3/VTTI_data/DSO/camera_forward.yaml', 'r') as f:
    calib = yaml.safe_load(f)

# Grab camera instrinsics and distortion coeffs
K = np.array(calib['camera_matrix']['data']).reshape(3,3)
print("Original instrinsics: \n", K)
D = np.array(calib['distortion_coefficients']['data']).reshape(1,8)
# print(D)
w = calib['image_width']
h = calib['image_height']
print(w,h)

# Compute undistortion maps
# Takes K and D (original intrinsics + distortion)
# Computes new camera matrix K - new_K
# 1: keep all pix, no sclaing
# roi: region of interest (keeping full images in this case) - ADJUSTING FROM 1 to ZERO***
new_k, roi = cv2.getOptimalNewCameraMatrix(K,D, (w,h), alpha = 0, newImgSize=(w,h))
print("New instrinsics: \n", new_k)
print("Still not exactly sure what the roi is \n", roi)

# Build lookup tables for maps
map1, map2 = cv2.initUndistortRectifyMap(K, D, None, new_k, (w,h), cv2.CV_16SC2)
# print("\nMap 1\n", map1, "\nMap 2\n", map2)

# Process all images
in_dir = "/home/daniel-choate/Datasets/VTTI_airfield/MSNV_0000_0000_0_250804_1830_00156/Images"
out_dir = "/home/daniel-choate/Datasets/VTTI_airfield/MSNV_0000_0000_0_250804_1830_00156/Images_PP"
os.makedirs(out_dir, exist_ok=True)


Original instrinsics: 
 [[1.61482951e+03 0.00000000e+00 1.23500441e+03]
 [0.00000000e+00 1.61997672e+03 6.96002777e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
2448 1376
New instrinsics: 
 [[1.43223878e+03 0.00000000e+00 1.23664616e+03]
 [0.00000000e+00 1.55486685e+03 6.96741068e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Still not exactly sure what the roi is 
 (0, 0, 2447, 1375)


In [3]:
print(calib)

{'camera_name': 'forward', 'camera_barcode': '', 'image_width': 2448, 'image_height': 1376, 'camera_matrix': {'rows': 3, 'cols': 3, 'data': [1614.82951, 0.0, 1235.00441, 0.0, 1619.97672, 696.002777, 0.0, 0.0, 1.0]}, 'distortion_model': 'rational_polynomial', 'distortion_coefficients': {'rows': 1, 'cols': 8, 'data': [49.0967308, 84.2090797, 5.78514307e-05, -0.000167231838, 25.2012578, 49.3039768, 95.7972418, 38.9384233]}, 'rectification_matrix': {'rows': 3, 'cols': 3, 'data': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]}, 'projection_matrix': {'rows': 3, 'cols': 4, 'data': [1614.82951, 0.0, 1235.00441, 0.0, 0.0, 1619.97672, 696.002777, 0.0, 0.0, 0.0, 1.0, 0.0]}}


In [4]:
# CROP AMOUNT FOR TIMESTAMP CORRECTION 
top_crop = 70

# Loop over and undistort all images 
for img_path in sorted(glob.glob(os.path.join(in_dir, "*.jpg"))):
    img = cv2.imread(img_path)
    # MAGIC LINE: applies undistortion mapping
    # Bilinear interpolation when resampling 
    undistorted = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR)
    # print(undistorted.shape)
    
    # Crop to ROI: because we changed alpha to zero # REMOVING FOR NOW
    # x,y,wc,hc = roi
    # und_cropped = undistorted[y+top_crop:y+hc, x:x+wc]
    
    # Convert to grayscale - IGNORE FOR NOW, NEED INITIAL RESULTS ****
    gray = cv2.cvtColor(undistorted, cv2.COLOR_BGR2GRAY)
    # Save the image - same name
    out_name = os.path.join(out_dir, os.path.basename(img_path))
    cv2.imwrite(out_name, gray) # CHANGE TO GRAY ONCE USING *****

print("Done")

Done


In [ ]:
# New intrinsics
fx = new_k[0][0]
fy = new_k[1][1]
cx = new_k[0][2]
cy = new_k[1][2]

in_w, in_h = w, h  # width and height of the undistorted buffer

# Adjust cy for top crop
cy_new = cy - top_crop
out_h = hc - top_crop  # new output height
out_w = wc             # width stays the same

# Write updated calib.txt for DSO
calib_txt = f"""Pinhole {fx} {fy} {cx} {cy_new} 0
{in_w} {in_h}
crop
{out_w} {out_h}
"""
with open(os.path.join(out_dir, "calib.txt"), "w") as f:
    f.write(calib_txt)

print("Done. Cropped top 70px and wrote updated calib.txt")

In [ ]:
# CREATE THE GAMMA FILE 

# Linear mapping: intensity i maps to i/255
gamma = np.linspace(0, 255, 256)

# Save as space-separated text file (one row)
np.savetxt("gamma.txt", gamma.reshape(1, -1), fmt="%.6f")
print("Saved gamma.txt with linear response")

In [ ]:
dir = 'DSO/vignette.png'
im = cv2.imread(dir)
new_w, new_h = 2448, 1376
img_new = cv2.resize(im, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
out_name = dir
cv2.imwrite(out_name, img_new)
print(im)

In [ ]:
import cv2
import numpy as np

height, width = 1376, 2448
vignette = np.ones((height, width), dtype=np.float32)  # flat vignette
vignette_16 = (vignette * 65535).astype(np.uint16)
cv2.imwrite("vignetteT.png", vignette_16)

In [ ]:
img = cv2.imread("vignetteT.png", cv2.IMREAD_UNCHANGED)
print(img.shape, img.dtype)